In [28]:
import os
os.chdir('/mnt/nas_siddharth/code_final/')

import cv2
import numpy as np
import copy
import os
import h5py
import pickle
import time
import glob

from src.utils import multicam_toolbox as mct
import matplotlib.pyplot as plt
import yaml
import scipy.io
from scipy.optimize import linear_sum_assignment

from tqdm import tqdm

In [29]:
def calc_dist(p1, p2):
    
    d = np.zeros([p1.shape[0],p2.shape[0]])
    d[:,:] = np.nan
    
    for i in range(p1.shape[0]):
        for j in range(p2.shape[0]):
            d[i,j] = np.sqrt(np.sum((p1[i,:]-p2[j,:])**2))

    d[np.isnan(d)] = 1e20
    
    return d

In [30]:
def check_performance(data_name, result_dir, kf_range=[0,1500]):
    with open('/mnt/nas_siddharth/tracking_gt/calib/config.yaml', 'r') as f:
        cfg = yaml.safe_load(f)
    ID = cfg['camera_id']

    annotation_file = f'/mnt/nas_siddharth/tracking_gt/annotation/{data_name}.pickle'
    modelout_file = glob.glob(os.path.join(result_dir, data_name + '.*', 'kp3d.pickle')) #added '.*'
    modelout_file = modelout_file[0]
    fnum_file = glob.glob(os.path.join(result_dir, data_name + '*/' + str(ID[0]) + '/frame_num.npy'))
    
    fnum_file = fnum_file[0]

    with open(annotation_file, 'rb') as f:
        anno = pickle.load(f)

    with open(modelout_file, 'rb') as f:
        mout = pickle.load(f)

    fnum = np.load(fnum_file)

    KF = np.array(anno['keyframe_fnum'])
    KF = KF[kf_range[0]:kf_range[1],0]

    X_pred = mout['kp3d']
    S_pred = mout['kp3d_score']

    cnt = 0
    R = []
    for i, kf in enumerate(KF):
        ii = i + kf_range[0]
        j = np.argwhere(fnum==kf).ravel()
        j = j[0]
        x_gt = anno['annotation'][ii]['p3d']
        skip_flag = anno['annotation'][ii]['skipped']

        if np.sum(skip_flag) > 0:
            #print(i, j, 'skipped')
            continue
        
        x_pred = np.mean(X_pred[:,j,5:7,:], axis=1)
        s_pred = np.nanmean(S_pred[:,j,:], axis=1)

        I_pred = np.argwhere(s_pred>0).ravel()
        I_gt = np.argwhere(np.logical_not(np.isnan(x_gt[:,0]))).ravel()

        x_pred = x_pred[I_pred,:]
        x_gt = x_gt[I_gt,:]
        d = calc_dist(x_pred, x_gt)
        row_ind, col_ind = linear_sum_assignment(d)

        r = np.zeros(4)

        tp = 0
        for i_sub in range(col_ind.shape[0]):
            if I_gt[col_ind[i_sub]] == I_pred[row_ind[i_sub]]:
                if d[row_ind[i_sub], col_ind[i_sub]] < 400:
                    tp += 1 # true positive
        
        n_gt = x_gt.shape[0]
        n_pred = x_pred.shape[0]

        for i_sub in range(4):
            if i_sub in I_pred.tolist() and i_sub not in I_gt.tolist():
                x_exit = np.array([5000, 0, 800])
                d_ = np.sqrt(np.sum((x_pred[I_pred==i_sub,:]-x_exit)**2))
                if d_ < 1000:
                    n_pred -= 1

        R.append([tp, n_gt, n_pred])
        #print(i,j,r)

    R = np.array(R)
    RR = np.sum(R, axis=0)
    TP = RR[0]
    FN = RR[1] - TP
    FP = RR[2] - TP

    precision = TP/(TP+FP)
    recall = TP/(TP+FN)

    print(TP, FN, FP, precision, recall)

    return TP, FN, FP, precision, recall

In [31]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [32]:
import pickle

with open('/mnt/nas_siddharth/code_final/results3D_sweep/gp1d_20211015_170000/MT0.65_PR0.30_TL0.05_TB36/gp1d_20211015_170000.2325-2705/kp3d.pickle', 'rb') as f:
    data = pickle.load(f)

print(data.keys())
print(data['kp3d'].shape)
print(len(data['joint_len']))


dict_keys(['kp3d', 'kp3d_score', 'kp3d_err', 'joint_len'])
(4, 9097, 17, 3)
4


In [33]:
import pickle

with open('/mnt/nas_siddharth/backup/results3D/gp1d_20211015_170000/kp3d.pickle', 'rb') as f:
    data = pickle.load(f)

print(data.keys())
print(data['kp3d'].shape)
print(len(data['joint_len']))


dict_keys(['kp3d', 'kp3d_score', 'kp3d_err', 'joint_len'])
(4, 84949, 17, 3)
4


In [ ]:
#result_dir = '/mnt/nas_siddharth/backup/results3D'
result_dir = './for_validation'

data_names = ['gp1d_20211015_170000']
KF_range = [[5900, 6650]]

result = np.zeros([len(data_names), 3])
for j, data_name in enumerate(data_names):
    print(data_name)
    TP, FN, FP, precision, recall = check_performance(data_name, result_dir, kf_range=KF_range[j])
    result[j,0] = TP
    result[j,1] = FN
    result[j,2] = FP

print('TOTAL:')
result_total = np.sum(result, axis=0)
TP = result_total[0]
FN = result_total[1]
FP = result_total[2]

precision = TP/(TP+FP)
recall = TP/(TP+FN)

print(TP, FN, FP, precision, recall)

gp1d_20211015_170000
2013 353 164 0.9246669728984842 0.8508030431107354
TOTAL:
2013.0 353.0 164.0 0.9246669728984842 0.8508030431107354


In [35]:
#result_dir = '/mnt/nas_siddharth/backup/results3D'
result_dir = './for_validation_2'
#data_names = [
#    'gp1d_20211015_170000'
#]
data_names = ['gp1d_20211015_170000']
KF_range = [[5900, 6650]]

result = np.zeros([len(data_names), 3])
for j, data_name in enumerate(data_names):
    print(data_name)
    TP, FN, FP, precision, recall = check_performance(data_name, result_dir, kf_range=KF_range[j])
    result[j,0] = TP
    result[j,1] = FN
    result[j,2] = FP

print('TOTAL:')
result_total = np.sum(result, axis=0)
TP = result_total[0]
FN = result_total[1]
FP = result_total[2]

precision = TP/(TP+FP)
recall = TP/(TP+FN)

print(TP, FN, FP, precision, recall)

gp1d_20211015_170000
2013 353 164 0.9246669728984842 0.8508030431107354
TOTAL:
2013.0 353.0 164.0 0.9246669728984842 0.8508030431107354
